In [2]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.4MB/s ta 0:00:011
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score
import pickle
import gc

In [19]:
os.listdir()

['bias_toxic_comment.ipynb',
 'sample_submission.csv',
 'README.md',
 'test.csv',
 '.git',
 'train.csv',
 '.ipynb_checkpoints']

In [20]:
BERT_MODEL = 'bert-base-uncased'
CASED = 'uncased' in BERT_MODEL
INPUT = '../input/jigsaw-bert-preprocessed-input/'
TEXT_COL = 'comment_text'
MAXLEN = 220

In [21]:
train = pd.read_csv('train.csv')

In [22]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 45 columns):
id                                     int64
target                                 float64
comment_text                           object
severe_toxicity                        float64
obscene                                float64
identity_attack                        float64
insult                                 float64
threat                                 float64
asian                                  float64
atheist                                float64
bisexual                               float64
black                                  float64
buddhist                               float64
christian                              float64
female                                 float64
heterosexual                           float64
hindu                                  float64
homosexual_gay_or_lesbian              float64
intellectual_or_learning_disability    float

In [24]:
pd.set_option('display.max_colwidth', -1)
print(train[11:12]['comment_text'])

11    This is a great story. Man. I wonder if the person who yelled "shut the fuck up!" at him ever heard it.
Name: comment_text, dtype: object


In [25]:
train = train.fillna(0)

In [26]:
train[train['target']>0.5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106438 entries, 4 to 1804872
Data columns (total 45 columns):
id                                     106438 non-null int64
target                                 106438 non-null float64
comment_text                           106438 non-null object
severe_toxicity                        106438 non-null float64
obscene                                106438 non-null float64
identity_attack                        106438 non-null float64
insult                                 106438 non-null float64
threat                                 106438 non-null float64
asian                                  106438 non-null float64
atheist                                106438 non-null float64
bisexual                               106438 non-null float64
black                                  106438 non-null float64
buddhist                               106438 non-null float64
christian                              106438 non-null float64
female   

In [27]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos to you for taking it on. Very impressive!,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on my site? When will you be releasing it?,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [28]:
from string import punctuation

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    
    punc_dict = {'.':'||Period||',
                 ',':'||Comma||',
                 '"':'||Quotation_Mark||',
                 ';':'||Semicolon||',
                 '!':'||Exclamation_Mark||',
                 '?':'||Question_Mark||',
                 '(':'||Left_Parentheses||',
                 ')':'||Right_Parentheses||',
                 '-':'||Dash||',
                 '\n':'||Return||',} 
    return punc_dict


In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def comment_to_words(comment):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(comment, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words


In [30]:
train.iloc[0]

id                                     59848                                                                                                
target                                 0                                                                                                    
comment_text                           This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
severe_toxicity                        0                                                                                                    
obscene                                0                                                                                                    
identity_attack                        0                                                                                                    
insult                                 0                                                                                                    
threat       

In [31]:
print(comment_to_words(train.iloc[0]['comment_text']))

['cool', 'like', 'would', 'want', 'mother', 'read', 'realli', 'great', 'idea', 'well', 'done']


In [13]:
import sys
package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.append(package_dir)

In [6]:
!conda install -c conda-forge/label/cf201901 pytorch-pretrained-bert < "y"

/bin/sh: y: No such file or directory


In [7]:
!pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 133kB 4.5MB/s ta 0:00:01
    100% |████████████████████████████████| 655kB 25.3MB/s ta 0:00:01
    100% |████████████████████████████████| 676.9MB 30kB/s  eta 0:00:01  4% |█▎                              | 28.1MB 21.5MB/s eta 0:00:31    5% |█▊                              | 36.3MB 36.0MB/s eta 0:00:18    21% |███████                         | 146.6MB 13.4MB/s eta 0:00:40    27% |████████▉                       | 187.9MB 20.7MB/s eta 0:00:24    31% |██████████                      | 212.1MB 17.2MB/s eta 0:00:27    32% |██████████▌                     | 222.0MB 14.8MB/s eta 0:00:31    58% |██████████████████▋             | 394.8MB 11.0MB/s eta 0:00:26    71% |██████████████████████▉         | 483.6MB 11.1MB/s eta 0:00:18    75% |████████████████████████▎       | 512.4MB 21.0MB/s eta 0:00:08    76% |████████████████████████▌       | 518.7MB 34.4MB/s eta 0:00:05    77% |████████████████████████▉       | 524.1MB 13.3MB/s eta 0:00:12    83% |██████

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
#import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig
from pytorch_pretrained_bert.modeling import BertModel


warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 1068278.48B/s]


In [25]:
#tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)


In [27]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [28]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

100%|██████████| 407873900/407873900 [00:37<00:00, 10922725.64B/s]


RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:51

In [ ]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'